In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
!pip install pyspark

/kaggle/input/jigsaw-regression-based-data/train_data_version1.csv
/kaggle/input/jigsaw-regression-based-data/train_data_version3.csv
/kaggle/input/jigsaw-regression-based-data/train_data_version2.csv
/kaggle/input/jigsaw-regression-based-data/FastText-jigsaw-256D/Jigsaw-Fasttext-Word-Embeddings-256D.bin.wv.vectors_ngrams.npy
/kaggle/input/jigsaw-regression-based-data/FastText-jigsaw-256D/Jigsaw-Fasttext-Word-Embeddings-256D.bin.wv.vectors_vocab.npy
/kaggle/input/jigsaw-regression-based-data/FastText-jigsaw-256D/Jigsaw-Fasttext-Word-Embeddings-256D.bin
/kaggle/input/jigsaw-regression-based-data/FastText-jigsaw-256D/Jigsaw-Fasttext-Word-Embeddings-256D.bin.syn1neg.npy
/kaggle/input/jigsaw-regression-based-data/FastText-jigsaw-100D/Jigsaw-Fasttext-Word-Embeddings.bin
/kaggle/input/jigsaw-regression-based-data/FastText-jigsaw-100D/Jigsaw-Fasttext-Word-Embeddings.bin.wv.vectors_ngrams.npy
     |████████████████████████████████| 281.4 MB 35 kB/s               
  Preparing metadata (setup.py

In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [3]:
sc=SparkContext(master="local")
spark=SparkSession(sc)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/19 01:44:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.catalog.listTables() # check whether there are tables in spark session

[]

In [5]:
pd_temp = pd.DataFrame(np.random.random(10)) # this is a pandas dataframe
spark_df=spark.createDataFrame(pd_temp) # spark dataframe image created locally 
spark.catalog.listTables() # as created locally will not find the spark dataframe

[]

In [6]:
spark_df.createOrReplaceTempView('random') # created a temp copy from local copy 
spark.catalog.listTables() # now one can see the spark dataframe in session

[Table(name='random', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [7]:
df_sql_spark=spark.sql("select * from random") # using spark.sql to fetch results from spark dataframe
df_pandas=df_sql_spark.toPandas() # spark dataframe converted to pandas dataframe

In [8]:
# directly importing csv file into spark
path="/kaggle/input/jigsaw-regression-based-data/train_data_version3.csv"
jigsaw=spark.read.csv(path,header=True)
jigsaw.show(2)
jigsaw.createOrReplaceTempView("jigsaw") # creating from local copy

+--------------------+----+
|                text|   y|
+--------------------+----+
|oh why dear johnn...|2.08|
|savant13 is a fat...|3.58|
+--------------------+----+
only showing top 2 rows



In [9]:
jigsaw_df=spark.table("jigsaw")
jigsaw_df=jigsaw_df.withColumn('x',jigsaw_df.y*2)
jigsaw_df.show(2)

+--------------------+----+----+
|                text|   y|   x|
+--------------------+----+----+
|oh why dear johnn...|2.08|4.16|
|savant13 is a fat...|3.58|7.16|
+--------------------+----+----+
only showing top 2 rows

